# Corretor Ortográfico em Python: aplicando técnicas de NLP

## 01. Explorando um projeto de NLP

### 04. Importando um corpus textual

In [1]:
with open("dados/artigos.txt") as f:
    artigos = f.read()

artigos[:500]

'\n\n\nimagem \n\nTemos a seguinte classe que representa um usuário no nosso sistema:\n\njava\n\nPara salvar um novo usuário, várias validações são feitas, como por exemplo: Ver se o nome só contém letras, [**o CPF só números**] e ver se o usuário possui no mínimo 18 anos. Veja o método que faz essa validação:\n\njava \n\nSuponha agora que eu tenha outra classe, a classe `Produto`, que contém um atributo nome e eu quero fazer a mesma validação que fiz para o nome do usuário: Ver se só contém letras. E aí? Vou'

### 05. Tokenização

In [ ]:
texto_exemplo = "Olá, tudo bem?"
tokens = texto_exemplo.split(" ")
print(len(tokens))
print(tokens)

3
['Olá,', 'tudo', 'bem?']


## 02. Utilizando NLTK para tokenizar um texto

### 02. Refinando a tokenização

[Natural Language Toolkit - NLTK](https://www.nltk.org/)

In [128]:
import nltk

lista_tokens = nltk.tokenize.word_tokenize(artigos)
lista_tokens[:30]

['imagem',
 'Temos',
 'a',
 'seguinte',
 'classe',
 'que',
 'representa',
 'um',
 'usuário',
 'no',
 'nosso',
 'sistema',
 ':',
 'java',
 'Para',
 'salvar',
 'um',
 'novo',
 'usuário',
 ',',
 'várias',
 'validações',
 'são',
 'feitas',
 ',',
 'como',
 'por',
 'exemplo',
 ':',
 'Ver']

### 04. Separando palavras de tokens

In [5]:
from typing import List

def separa_palavras(lista_tokens: List[str]) -> List[str]:
    lista_palavras = list()
    for token in lista_tokens:
        if token.isalpha():
            lista_palavras.append(token)
    return lista_palavras

### 05. Contando palavras do Corpus

In [8]:
lista_palavras = separa_palavras(lista_tokens)
len(lista_palavras)

403104

### 06. Normalização

In [9]:
def normalizacao(lista_palavras: List[str]) -> List[str]:
    lista_normalizada = list()
    for palavra in lista_palavras:
        lista_normalizada.append(palavra.lower())
    return lista_normalizada

In [10]:
lista_normalizada = normalizacao(lista_palavras)
lista_normalizada[:5]

['imagem', 'temos', 'a', 'seguinte', 'classe']

### 07. Tipos de palavras

In [11]:
print(len(set(lista_normalizada)))

18465


## 03. Desenvolvendo e testando o corretor

### 03. Fatiando strings

In [34]:
palavra_exemplo = "lgica"

def gerador_palavras(palavra: str) -> List[str]:
    fatias = list()
    for i in range(len(palavra) + 1):
        fatias.append((palavra[:i], palavra[i:]))
    print(fatias)

In [35]:
gerador_palavras(palavra_exemplo)

[('', 'lgica'), ('l', 'gica'), ('lg', 'ica'), ('lgi', 'ca'), ('lgic', 'a'), ('lgica', '')]


### 04. Operação de inserção

In [36]:
from typing import Tuple

def insere_letras(fatias: List[Tuple[str, str]]) -> List[str]:
    novas_palavras = list()
    letras = "abcdefghijklmnopqrstuvwxyzáâàãéêèẽíîìĩóôòõúûùũç"
    for E, D in fatias:
        for letra in letras:
            novas_palavras.append(E + letra + D)
    return novas_palavras

In [97]:
def gerador_palavras(palavra: str) -> List[str]:
    fatias = list()
    for i in range(len(palavra) + 1):
        fatias.append((palavra[:i], palavra[i:]))
    palavras_geradas = insere_letras(fatias)
    return palavras_geradas

In [38]:
gerador_palavras(palavra_exemplo)[:10]

['algica',
 'blgica',
 'clgica',
 'dlgica',
 'elgica',
 'flgica',
 'glgica',
 'hlgica',
 'ilgica',
 'jlgica']

### 06. Construindo a função corretor

In [98]:
def corretor(palavra: str) -> str:
    palavras_geradas = gerador_palavras(palavra)
    palavra_correta = max(palavras_geradas, key=probabilidade)
    return palavra_correta

### 07. Probabilidade das palavras geradas

In [18]:
frequencia = nltk.FreqDist(lista_normalizada)
total_palavras = len(lista_normalizada)

In [92]:
def probabilidade(palavra_gerada: str):
    return frequencia[palavra_gerada]/total_palavras

In [96]:
def corretor(palavra: str) -> str:
    palavras_geradas = gerador_palavras(palavra)
    palavra_correta = max(palavras_geradas, key=probabilidade)
    return palavra_correta

In [93]:
corretor(palavra_exemplo)

'lógica'

## 04. Avaliando a qualidade do corretor

### 02. Preparando dados de teste

In [86]:
def cria_dados_teste(nome_arquivo: str) -> List[str]:
    lista_palavras_teste = list()
    f = open(nome_arquivo, "r")
    for linha in f:
        correta, errada = linha.split(" ")
        lista_palavras_teste.append((correta, errada.replace("\n", "")))
    f.close()
    return lista_palavras_teste

In [87]:
lista_teste = cria_dados_teste("./dados/palavras.txt")
lista_teste[:10]

[('podemos', 'pyodemos'),
 ('esse', 'esje'),
 ('já', 'jrá'),
 ('nosso', 'nossov'),
 ('são', 'sãêo'),
 ('dos', 'dosa'),
 ('muito', 'muifo'),
 ('imagem', 'iômagem'),
 ('sua', 'ósua'),
 ('também', 'tambéùm')]

### 03. Avaliando o corretor

In [75]:
def avaliador(testes: List[str]) -> None:
    numero_palavras = len(testes)
    acertou = 0
    for correta, errada in testes:
        palavra_corrigida = corretor(errada)
        if palavra_corrigida == correta:
            acertou += 1
    taxa_acerto = round(acertou*100/numero_palavras, 2)
    print(f"{taxa_acerto}% de {numero_palavras} palavras")

In [99]:
avaliador(lista_teste)

1.08% de 186 palavras


## 05. Incrementando o corretor

### 03. Implementando o delete de caracteres

In [57]:
def deletando_caracteres(fatias: List[Tuple[str, str]]) -> List[str]:
    novas_palavras = list()
    for E, D in fatias:
        novas_palavras.append(E + D[1:])
    return novas_palavras

### 05. Avaliando o novo corretor

In [100]:
def gerador_palavras(palavra: str) -> List[str]:
    fatias = list()
    for i in range(len(palavra) + 1):
        fatias.append((palavra[:i], palavra[i:]))
    palavras_geradas = insere_letras(fatias)
    palavras_geradas += deletando_caracteres(fatias)
    return palavras_geradas

In [133]:
palavra_exemplo = "lóigica"
gerador_palavras(palavra_exemplo)[::-1][:10]

['lóigiac',
 'lóigcia',
 'lóiigca',
 'lógiica',
 'liógica',
 'óligica',
 'lóigicaç',
 'lóigicaũ',
 'lóigicaù',
 'lóigicaû']

In [132]:
avaliador(lista_teste)

76.34% de 186 palavras


## 06. Corrigindo os principais erros de digitação

### 03. Implementando a troca de letras

In [104]:
def insere_letras(fatias: List[Tuple[str, str]]) -> List[str]:
    novas_palavras = list()
    letras = "abcdefghijklmnopqrstuvwxyzáâàãéêèẽíîìĩóôòõúûùũç"
    for E, D in fatias:
        for letra in letras:
            novas_palavras.append(E + letra + D)
    return novas_palavras


def deletando_caracteres(fatias: List[Tuple[str, str]]) -> List[str]:
    novas_palavras = list()
    for E, D in fatias:
        novas_palavras.append(E + D[1:])
    return novas_palavras


def troca_letra(fatias: List[Tuple[str, str]]) -> List[str]:
    novas_palavras = list()
    letras = "abcdefghijklmnopqrstuvwxyzáâàãéêèẽíîìĩóôòõúûùũç"
    for E, D in fatias:
        for letra in letras:
            novas_palavras.append(E + letra + D[1:])
    return novas_palavras


def gerador_palavras(palavra: str) -> List[str]:
    fatias = list()
    for i in range(len(palavra) + 1):
        fatias.append((palavra[:i], palavra[i:]))
    palavras_geradas = (insere_letras(fatias))
    palavras_geradas.extend(deletando_caracteres(fatias))
    palavras_geradas.extend(troca_letra(fatias))
    return palavras_geradas

In [109]:
palavra_exemplo = "lígica"
palavras_geradas = gerador_palavras(palavra_exemplo)
palavras_geradas[:10]

['alígica',
 'blígica',
 'clígica',
 'dlígica',
 'elígica',
 'flígica',
 'glígica',
 'hlígica',
 'ilígica',
 'jlígica']

### 06. Codando a inversão de letras

In [121]:
def inverte_letra(fatias: List[Tuple[str, str]]) -> List[str]:
    novas_palavras = list()
    for E, D in fatias:
        if len(D) > 1:
            novas_palavras.append(E + D[1] + D[0] + D[2:])
    return novas_palavras

In [119]:
def gerador_palavras(palavra: str) -> List[str]:
    fatias = list()
    for i in range(len(palavra) + 1):
        fatias.append((palavra[:i], palavra[i:]))
    palavras_geradas = (insere_letras(fatias))
    palavras_geradas.extend(deletando_caracteres(fatias))
    palavras_geradas.extend(troca_letra(fatias))
    palavras_geradas.extend(inverte_letra(fatias))
    return palavras_geradas

In [126]:
gerador_palavras("lgóica")[::-1][:5]

['lgóiac', 'lgócia', 'lgióca', 'lógica', 'glóica']

In [127]:
avaliador(lista_teste)

76.34% de 186 palavras


## 07. Criando um corretor turbinado

### 02. Palavras desconhecidas ao vocabulário

In [136]:
def avaliador(testes: List[str], vocabulario: List[str]) -> None:
    numero_palavras = len(testes)
    acertou = 0
    desconhecida = 0
    for correta, errada in testes:
        palavra_corrigida = corretor(errada)
        if palavra_corrigida == correta:
            acertou += 1
        else:
            desconhecida += (palavra_corrigida not in vocabulario)
    taxa_acerto = round(acertou*100/numero_palavras, 2)
    taxa_desconhecida = round(desconhecida*100/numero_palavras, 2)
    print(f"{taxa_acerto}% de {numero_palavras} palavras, {taxa_desconhecida}% palavras desconhecidas")

In [137]:
vocabulario = set(lista_normalizada)
avaliador(lista_teste, vocabulario)

76.34% de 186 palavras, 16.13% palavras desconhecidas


### 03. Turbinando o gerador de palavras

In [139]:
def gerador_turbinado(palavras_geradas: List[str]) -> List[str]:
    novas_palavras = list()
    for palavra in palavras_geradas:
        novas_palavras.extend(gerador_palavras(palavra))
    return novas_palavras

In [140]:
palavra = "lóiigica"
palavras_geradas = gerador_turbinado(gerador_palavras(palavra))
"lógica" in palavras_geradas

True

In [142]:
len(palavras_geradas)

787396

### 04. Escolhendo os melhores candidatos

In [147]:
def novo_corretor(palavra: str) -> str:
    palavras_geradas = gerador_palavras(palavra)
    palavras_geradas.extend(gerador_turbinado(palavras_geradas))
    candidatos = [palavra]
    for palavra in set(palavras_geradas):
        if palavra in vocabulario:
            candidatos.append(palavra)
    # print(len(candidatos))
    palavra_correta = max(candidatos, key=probabilidade)
    return palavra_correta

In [146]:
novo_corretor("lóiigica")

2


'lógica'

## 08. Avaliando e interpretando o erro do corretor turbinado

### 02. Avaliando o resultado dos dois corretores

In [156]:
def avaliador(testes: List[str], vocabulario: List[str]) -> None:
    numero_palavras = len(testes)
    acertou = 0
    desconhecida = 0
    for correta, errada in testes:
        palavra_corrigida = novo_corretor(errada)
        desconhecida += (palavra_corrigida not in vocabulario)
        if palavra_corrigida == correta:
            acertou += 1
        # else:
            # print(f"{errada} - {corretor(errada)} - {palavra_corrigida}")
    taxa_acerto = round(acertou*100/numero_palavras, 2)
    taxa_desconhecida = round(desconhecida*100/numero_palavras, 2)
    print(f"{taxa_acerto}% de {numero_palavras} palavras, {taxa_desconhecida}% palavras desconhecidas")

In [157]:
avaliador(lista_teste, vocabulario)

55.38% de 186 palavras, 2.69% palavras desconhecidas


### 03. Avaliando o resultado dos dois corretores cont

In [154]:
def avaliador(testes: List[str], vocabulario: List[str]) -> None:
    numero_palavras = len(testes)
    acertou = 0
    desconhecida = 0
    for correta, errada in testes:
        palavra_corrigida = corretor(errada)
        desconhecida += (palavra_corrigida not in vocabulario)
        if palavra_corrigida == correta:
            acertou += 1
    taxa_acerto = round(acertou*100/numero_palavras, 2)
    taxa_desconhecida = round(desconhecida*100/numero_palavras, 2)
    print(f"{taxa_acerto}% de {numero_palavras} palavras, {taxa_desconhecida}% palavras desconhecidas")

In [155]:
avaliador(lista_teste, vocabulario)

76.34% de 186 palavras, 16.13% palavras desconhecidas


In [158]:
palavra = "lgica"
print(corretor(palavra))
print(novo_corretor(palavra))

lógica
fica
